Capstone Week 1
Introdution:
Suppose one were to open a new fast food restaurant in Chicago,IL. The main intent of this restaurant would be speed and ease of access. There are many mass transit points in Chicago and being close to many of them would be a major bonus to their commuter business. The trains would be most beneficial as they carry the most commuters, but busses can't be ignored either. The final point would be distance from competition. This could even out as foot traffic near competitors can increase business but being the only place in an area can give a boost from demand versus supply.

Data:
Using a point in downtown Chicago, we will pull in the transit lines, the fast food and major points of interest to find the best location for said restaurant. Chicago works on a grid system, 0,0 is the corner of State st and Madison st and is fairly close to the center of downtown.

In [1]:
import requests
import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import folium

In [83]:
CLIENT_ID = 'B4U2MZYSR1BHH4SH5B5K1NEQRYKBRFRVXFBBTEIUAM2WPAK3' # your Foursquare ID
CLIENT_SECRET = 'ZNRMEUKU45XWHJDYQBPA2WQVCPVPYFVONLAPPY0VV12XJB4E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 250

In [84]:
address = '50 West Madison St, Chicago, IL'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.88133175 -87.6422359802366


In [161]:
radius = 750


In [179]:
search_query = 'CTA'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [180]:
search_query = 'Fast food'
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [182]:
CTAresults = requests.get(url).json()

In [183]:
FFresults = requests.get(url2).json()

In [184]:
CTAven = CTAresults['response']['venues']
FFven = FFresults['response']['venues']

In [185]:
CTAdf = pd.json_normalize(CTAven)
FFdf = pd.json_normalize(FFven)

In [187]:
CTAbus = CTAdf[CTAdf['name'].str.contains("Bus")]
CTAtrain = CTAdf[~CTAdf['name'].str.contains("Bus|bus")]

In [197]:
def filter_dataframe(df):

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = df.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
            try:
                categories_list = row['categories']
            except:
                categories_list = row['venue.categories']
        
            if len(categories_list) == 0:
                return None
            else:
                return categories_list[0]['name']

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    return dataframe_filtered

In [198]:
Busdf = filter_dataframe(CTAbus)
Traindf= filter_dataframe(CTAtrain)
FastFooddf = filter_dataframe(FFdf)


In [199]:
Busdf

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
1,CTA Bus Stop 456,Bus Station,Madison & Jefferson,Westbound,41.881718,-87.642377,"[{'label': 'display', 'lat': 41.88171776726318...",44,60661,US,Chicago,IL,United States,"[Madison & Jefferson (Westbound), Chicago, IL ...",4e959e7ef5b90ceed071b293
2,CTA Bus Stop 5009,Bus Station,Canal & Adams,Northbound,41.879236,-87.639554,"[{'label': 'display', 'lat': 41.87923550008927...",322,60606,US,Chicago,IL,United States,"[Canal & Adams (Northbound), Chicago, IL 60606...",4dc1df3ed1643ed27a4346dd
5,CTA Bus Stop 442,Bus Station,Washington & Jefferson,Eastbound,41.883423,-87.642984,"[{'label': 'display', 'lat': 41.88342266771971...",240,60661,US,Chicago,IL,United States,"[Washington & Jefferson (Eastbound), Chicago, ...",4f022358e300fdc04b0c6473
7,CTA Bus Stop 5516,Bus Station,Clinton & Madison,Southbound,41.881967,-87.641265,"[{'label': 'display', 'lat': 41.88196716412095...",107,60606,US,Chicago,IL,United States,"[Clinton & Madison (Southbound), Chicago, IL 6...",4c1d874fb4e62d7fedffdc93
8,CTA Bus Stop 455,Bus Station,Madison & Clinton,Westbound,41.881786,-87.641199,"[{'label': 'display', 'lat': 41.88178600442451...",99,60661,US,Chicago,IL,United States,"[Madison & Clinton (Westbound), Chicago, IL 60...",4bbc90f2e4529521140556a4
9,CTA Bus Stop 5007,Bus Station,Clinton & Monroe,Southbound,41.880696,-87.641274,"[{'label': 'display', 'lat': 41.88069580059553...",106,60661,US,Chicago,IL,United States,"[Clinton & Monroe (Southbound), Chicago, IL 60...",4ed433d38b81e32244cbefa3
11,CTA Bus Stop 1108,Bus Station,Adams & S Wacker,Westbound,41.879144,-87.636304,"[{'label': 'display', 'lat': 41.87914358634876...",548,60606,US,Chicago,IL,United States,"[Adams & S Wacker (Westbound), Chicago, IL 606...",4be16527ea6e20a185fb9267
12,CTA Bus Stop 5010,Bus Station,Canal & Monroe,Northbound,41.880322,-87.639506,"[{'label': 'display', 'lat': 41.88032164557818...",252,60661,US,Chicago,IL,United States,"[Canal & Monroe (Northbound), Chicago, IL 6066...",4dd306487d8ba0f1bdf1c124
13,CTA Bus Stop 5027,Bus Station,Canal & Madison,Northbound,41.882127,-87.639549,"[{'label': 'display', 'lat': 41.88212692094602...",239,60606,US,Chicago,IL,United States,"[Canal & Madison (Northbound), Chicago, IL 606...",4bec08ee415e20a10dfbe6bb
14,CTA Bus Stop 15288,Bus Station,Adams & Clinton,Westbound,41.879371,-87.640922,"[{'label': 'display', 'lat': 41.87937105973175...",243,60606,US,Chicago,IL,United States,"[Adams & Clinton (Westbound), Chicago, IL 6060...",4da62b574b22737295bb8159


In [200]:
Traindf

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,CTA - Washington/Wells,Metro Station,100 N Wells St,at W Washington St,41.882437,-87.633915,"[{'label': 'entrance', 'lat': 41.882153, 'lng'...",700,60606,US,Chicago,IL,United States,"[100 N Wells St (at W Washington St), Chicago,...",4a6792c8f964a520b4c91fe3
3,CTA - Clinton/Lake,Train Station,540 W. Lake St.,at Clinton St.,41.885626,-87.641474,"[{'label': 'display', 'lat': 41.88562594874706...",482,60661,US,Chicago,IL,United States,"[540 W. Lake St. (at Clinton St.), Chicago, IL...",4aac0285f964a520955b20e3
4,CTA - Clinton,Metro Station,426 S Clinton St,at Tilden,41.875943,-87.640907,"[{'label': 'display', 'lat': 41.87594287325161...",609,60607,US,Chicago,IL,United States,"[426 S Clinton St (at Tilden), Chicago, IL 606...",4a95f0a4f964a520912520e3
6,CTA - Morgan,Train Station,1000 W Lake St,at Morgan St.,41.885634,-87.651924,"[{'label': 'display', 'lat': 41.88563397729588...",934,60607,US,Chicago,IL,United States,"[1000 W Lake St (at Morgan St.), Chicago, IL 6...",4ea6eae48b814c68ad191bd2
10,CTA - Quincy/Wells,Train Station,220 S Wells St,at W Quincy St,41.879755,-87.633925,"[{'label': 'entrance', 'lat': 41.879334, 'lng'...",710,60606,US,Chicago,IL,United States,"[220 S Wells St (at W Quincy St), Chicago, IL ...",4b60d577f964a52061fc29e3
29,CTA - LaSalle,Metro Station,150 W Congress Pkwy,at LaSalle St.,41.878093,-87.635227,"[{'label': 'display', 'lat': 41.8780930901371,...",683,60605,US,Chicago,IL,United States,"[150 W Congress Pkwy (at LaSalle St.), Chicago...",4e9e53b86c25364827883f23


In [201]:
FastFooddf

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,IFT - Institute Of Food Technologists,Office,525 W Van Buren St Ste 1000,41.879794,-87.639413,"[{'label': 'display', 'lat': 41.87979402699733...",289,60607,US,Chicago,IL,United States,"[525 W Van Buren St Ste 1000, Chicago, IL 6060...",NaN,4daf185ff7b17488f49f85bc
1,Chicago Union Station Food Court,Food Court,225 S Canal St,41.879339,-87.638738,"[{'label': 'display', 'lat': 41.879339217638, ...",365,60606,US,Chicago,IL,United States,"[225 S Canal St (at Adams St.), Chicago, IL 60...",at Adams St.,4c489f3a19fde21e8b460c76
2,Ogilvie Food Court,Food Court,500 W Madison St,41.882125,-87.640672,"[{'label': 'display', 'lat': 41.88212517500138...",156,60661,US,Chicago,IL,United States,[500 W Madison St (at Ogilvie Transportation C...,at Ogilvie Transportation Center,4d0ff93ea3d9721e414fd2fd
3,Fast Track,Diner,629 W Lake St,41.885470,-87.644071,"[{'label': 'display', 'lat': 41.88546951693589...",485,60661,US,Chicago,IL,United States,"[629 W Lake St (at Desplaines St.), Chicago, I...",at Desplaines St.,4b8c6861f964a52045cf32e3
4,Bridgeport Pasty - Food Truck,Food Truck,NaN,41.879691,-87.636715,"[{'label': 'display', 'lat': 41.87969136866029...",492,60608,US,Chicago,IL,United States,"[Chicago, IL 60608, United States]",NaN,4e8c8fc793adecfdb971a040
5,FASTSIGNS,Design Studio,180 N Wacker Dr,41.885165,-87.636970,"[{'label': 'display', 'lat': 41.88516514544391...",610,60606,US,Chicago,IL,United States,"[180 N Wacker Dr, Chicago, IL 60606, United St...",NaN,4fe486d74c7fada8d19629f0
6,Willis Tower,Building,233 S Wacker Dr,41.878751,-87.635780,NaN,607,60606,US,Chicago,IL,United States,[233 S Wacker Dr (btwn W Adams St & W Jackson ...,btwn W Adams St & W Jackson Blvd,4a61162ff964a520d0c11fe3
7,Fooda Cafe,Café,NaN,41.882236,-87.642500,"[{'label': 'display', 'lat': 41.882236, 'lng':...",103,60661,US,Chicago,IL,United States,"[Chicago, IL 60661, United States]",NaN,57a9d484498e86d7d10bbe20
8,Gold Coast Dogs,Hot Dog Joint,500 W Madison St,41.881825,-87.640582,"[{'label': 'display', 'lat': 41.88182498429387...",147,60661,US,Chicago,IL,United States,[500 W Madison St (Canal (at Ogilvie Transport...,Canal (at Ogilvie Transportation Center),4a9eacc0f964a520df3a20e3
9,Just Food For Dogs,Pet Store,"901 W Jackson Blvd, Inside GoodVets West Loop",41.877702,-87.649925,"[{'label': 'display', 'lat': 41.87770243612904...",754,60607,US,Chicago,IL,United States,"[901 W Jackson Blvd, Inside GoodVets West Loop...",NaN,5f04407a19d98c15b6fe37ab


In [ ]:
|